In [2]:
!pip install pytorch torchvision torchaudio


  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Obtaining dependency information for torchvision from https://files.pythonhosted.org/packages/f9/e6/3c821e7417acd82df89e39f09156ce80d58817b5b4b1ac5453b522bc5dd4/torchvision-0.16.2-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for torchaudio from https://files.pythonhosted.org/packages/c9/c0/b738db223b85c0096e2c3b1aaa647419f9af68331f8ad09dba6a2d38136c/torchaudio-2.1.2-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for torch==2.1.2 from https://files.pythonhosted.org/packages/e4/ae/2ad8820045b6631965750435f28583e80905b8273d57cf026163b51323ee/torch-2.1.2-cp311-cp311-win_amd64.whl.metadata
   ---------------------------------------- 0.0/1.1 MB ? eta -:--:--
   - -------------------------------------- 0.0/1.1 MB 1.3 MB/s eta 0:00:01
   - -------------------------------------- 0.0/1.1 MB 653.6 kB/s eta 0:00:02
   --- --------------------------

  error: subprocess-exited-with-error
  
  python setup.py bdist_wheel did not run successfully.
  exit code: 1
  
  [6 lines of output]
  Traceback (most recent call last):
    File "<string>", line 2, in <module>
    File "<pip-setuptools-caller>", line 34, in <module>
    File "C:\Users\AL\AppData\Local\Temp\pip-install-n12wl7ul\pytorch_c72a16b1c82542aaa1a7afe23a91a385\setup.py", line 15, in <module>
      raise Exception(message)
  Exception: You tried to install "pytorch". The package named for PyTorch is "torch"
  [end of output]
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for pytorch
ERROR: Could not build wheels for pytorch, which is required to install pyproject.toml-based projects


In [3]:
!pip install torch torchvision torchaudio

  Obtaining dependency information for torch from https://files.pythonhosted.org/packages/e4/ae/2ad8820045b6631965750435f28583e80905b8273d57cf026163b51323ee/torch-2.1.2-cp311-cp311-win_amd64.whl.metadata
  Using cached torch-2.1.2-cp311-cp311-win_amd64.whl.metadata (26 kB)
  Obtaining dependency information for torchvision from https://files.pythonhosted.org/packages/f9/e6/3c821e7417acd82df89e39f09156ce80d58817b5b4b1ac5453b522bc5dd4/torchvision-0.16.2-cp311-cp311-win_amd64.whl.metadata
  Using cached torchvision-0.16.2-cp311-cp311-win_amd64.whl.metadata (6.6 kB)
  Obtaining dependency information for torchaudio from https://files.pythonhosted.org/packages/c9/c0/b738db223b85c0096e2c3b1aaa647419f9af68331f8ad09dba6a2d38136c/torchaudio-2.1.2-cp311-cp311-win_amd64.whl.metadata
  Using cached torchaudio-2.1.2-cp311-cp311-win_amd64.whl.metadata (6.4 kB)
Using cached torch-2.1.2-cp311-cp311-win_amd64.whl (192.3 MB)
Using cached torchvision-0.16.2-cp311-cp311-win_amd64.whl (1.1 MB)
Using cached

In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models


In [5]:
# Paths to your training and validation datasets
train_data_path = "training_data"
val_data_path = "validation_data"


In [6]:
# Define transforms for data augmentation and normalization
train_transform = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1, hue=0.1),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

val_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [7]:
# Create datasets
train_dataset = datasets.ImageFolder(train_data_path, transform=train_transform)
val_dataset = datasets.ImageFolder(val_data_path, transform=val_transform)


In [8]:
# Create data loaders
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=4)


In [9]:
# Initialize the model with pretrained weights
model = models.resnet34(pretrained=True)

D:\Anaconda\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
D:\Anaconda\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet34_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet34_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet34-b627a593.pth" to C:\Users\AL/.cache\torch\hub\checkpoints\resnet34-b627a593.pth
100%|█████████████████████████████████████████████████████████████████████████████| 83.3M/83.3M [01:20<00:00, 1.09MB/s]


In [10]:
# Freeze all layers in the network
for param in model.parameters():
    param.requires_grad = False

In [11]:
# Unfreeze the last few layers for fine-tuning
for param in model.layer4.parameters():
    param.requires_grad = True

In [12]:
# Replace the final fully connected layer with a new one (unfrozen by default)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 3)  # Assuming 3 classes

In [13]:
# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD([
    {'params': model.layer4.parameters(), 'lr': 0.001},
    {'params': model.fc.parameters(), 'lr': 0.01}
], momentum=0.9)

In [14]:
# Optional: Learning rate scheduler
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

In [15]:
# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    # Print loss every epoch
    print(f'Epoch {epoch + 1}/{num_epochs}, Training Loss: {running_loss/len(train_loader):.4f}')

    # Decay Learning Rate
    scheduler.step()

    # Validation loop
    model.eval()
    total_correct = 0
    total_samples = 0
    for inputs, labels in val_loader:
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        total_samples += labels.size(0)
        total_correct += (predicted == labels).sum().item()

    accuracy = total_correct / total_samples
    print(f'Epoch {epoch + 1}/{num_epochs}, Validation Accuracy: {accuracy:.4f}')

Epoch 1/10, Training Loss: 0.5296
Epoch 1/10, Validation Accuracy: 0.9293
Epoch 2/10, Training Loss: 0.3817
Epoch 2/10, Validation Accuracy: 0.8737
Epoch 3/10, Training Loss: 0.3522
Epoch 3/10, Validation Accuracy: 0.9394
Epoch 4/10, Training Loss: 0.3518
Epoch 4/10, Validation Accuracy: 0.9444
Epoch 5/10, Training Loss: 0.3456
Epoch 5/10, Validation Accuracy: 0.9192
Epoch 6/10, Training Loss: 0.3345
Epoch 6/10, Validation Accuracy: 0.9242
Epoch 7/10, Training Loss: 0.2345
Epoch 7/10, Validation Accuracy: 0.9545
Epoch 8/10, Training Loss: 0.2199
Epoch 8/10, Validation Accuracy: 0.9545
Epoch 9/10, Training Loss: 0.1924
Epoch 9/10, Validation Accuracy: 0.9545
Epoch 10/10, Training Loss: 0.1864
Epoch 10/10, Validation Accuracy: 0.9495


In [16]:
# Save the trained model
torch.save(model.state_dict(), 'your_trained_model.pth')

## Grad-CAM to visualize the activations of your model on a given image:

In [4]:
!pip install opencv-python

  Obtaining dependency information for opencv-python from https://files.pythonhosted.org/packages/c7/ec/9dabb6a9abfdebb3c45b0cc52dec901caafef2b2c7e7d6a839ed86d81e91/opencv_python-4.9.0.80-cp37-abi3-win_amd64.whl.metadata
   ---------------------------------------- 0.0/38.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/38.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/38.6 MB 960.0 kB/s eta 0:00:41
   ---------------------------------------- 0.1/38.6 MB 648.1 kB/s eta 0:01:00
   ---------------------------------------- 0.1/38.6 MB 930.9 kB/s eta 0:00:42
   ---------------------------------------- 0.2/38.6 MB 1.0 MB/s eta 0:00:37
   ---------------------------------------- 0.2/38.6 MB 1.3 MB/s eta 0:00:31
   ---------------------------------------- 0.3/38.6 MB 1.2 MB/s eta 0:00:33
   ---------------------------------------- 0.3/38.6 MB 1.3 MB/s eta 0:00:31
   ---------------------------------------- 0.4/38.6 MB 1.3 MB/s eta 0:00:29
   --------------

In [8]:
import torch
import torch.nn as nn
from torchvision import models, transforms
from PIL import Image
import cv2
import numpy as np

# Load the trained model
model = models.resnet34(weights=None)  # Replace pretrained=False with weights=None
num_classes = 3
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, num_classes)

# Load the trained weights
model.load_state_dict(torch.load('your_trained_model.pth', map_location=torch.device('cpu')))
model.eval()


class GradCAM:
    def __init__(self, model, target_layer):
        self.model = model
        self.target_layer = target_layer
        self.feature = None
        self.gradient = None
        self.register_hooks()

    def hook_fn_forward(self, module, input, output):
        self.feature = output

    def hook_fn_backward(self, module, grad_in, grad_out):
        self.gradient = grad_out[0]

    def register_hooks(self):
        # Use register_full_backward_hook
        self.target_layer.register_forward_hook(self.hook_fn_forward)
        self.target_layer.register_full_backward_hook(self.hook_fn_backward)

    def __call__(self, input_image):
        # Forward pass
        self.model.zero_grad()
        output = self.model(input_image)

        # Backward pass to get the gradient
        target = torch.argmax(output)
        output[:, target].backward()

        # Get the gradient of the target layer
        gradient = self.gradient

        # Global average pooling on gradients
        weights = torch.mean(gradient, dim=(1, 2), keepdim=True)

        # Weighted sum of feature maps
        cam = torch.sum(weights * self.feature, dim=1)[0]  # Sum over channels and select first sample

        # Convert to numpy and apply ReLU
        cam = cam.cpu().data.numpy()
        cam = np.maximum(cam, 0)

        # Normalize the CAM
        cam = cam - np.min(cam)
        cam = cam / np.max(cam)

        return cam

def get_grad_cam_image(model, image_path, target_layer, save_path):
    # Load and preprocess the image
    image = Image.open(image_path)
    preprocess = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    input_image = preprocess(image).unsqueeze(0)

    # Create Grad-CAM instance and generate CAM
    grad_cam = GradCAM(model=model, target_layer=target_layer)
    cam = grad_cam(input_image)

    # Normalize the CAM and convert to 8-bit format
    cam = cam - np.min(cam)
    cam = cam / np.max(cam)
    cam = np.uint8(255 * cam)

    # Resize the CAM and convert it to a heatmap
    original_image = np.array(image.resize((224, 224)))
    cam_resized = cv2.resize(cam, (original_image.shape[1], original_image.shape[0]))
    heatmap = cv2.applyColorMap(cam_resized, cv2.COLORMAP_JET)
    heatmap = cv2.cvtColor(heatmap, cv2.COLOR_BGR2RGB)

    # Overlay the heatmap on the original image
    result = heatmap * 0.4 + original_image * 0.6
    result = result.astype('uint8')

    # Convert to PIL image and save
    result_pil = Image.fromarray(result)
    result_pil.save(save_path)

    return result_pil

# Specify the paths
image_path = 'image510.png'
save_path = 'grad_cam_result3.png'

# Enable gradient tracking
torch.set_grad_enabled(True)

# Choose a target layer (you can experiment with different layers)
target_layer = model.layer4[-1].conv2

# Get the Grad-CAM result and save the image
result_image = get_grad_cam_image(model, image_path, target_layer, save_path)

# Show the image
result_image.show()    



In [9]:
import torch
from torchvision import models, transforms
from PIL import Image

# Define the path to your image and model
image_path = "image510.png"  # Change to the path of your new image
model_path = 'your_trained_model.pth'  # Change to the path of your saved model

# Define the same transforms that you used for your validation dataset
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Define a function for image prediction
def predict_image(image_path, model_path):
    # Load the image
    image = Image.open(image_path)
    image = transform(image).unsqueeze(0)  # Add batch dimension

    # Load the trained model
    model = models.resnet34(pretrained=False)
    num_ftrs = model.fc.in_features
    model.fc = nn.Linear(num_ftrs, 3)  # Assuming 3 classes
    model.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))
    model.eval()

    # Predict the class
    with torch.no_grad():
        outputs = model(image)
        _, predicted = torch.max(outputs, 1)
    
    # You can add a class index-to-name mapping based on your dataset
    class_names = ['dosa', 'idly', 'vada']  # Replace with your actual class names
    predicted_class = class_names[predicted[0]]

    return predicted_class

# Call the function and print the prediction
predicted_class = predict_image(image_path, model_path)
print(f"The model predicts this image as: {predicted_class}")


The model predicts this image as: idly
